In [107]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math
from datetime import datetime, timedelta
from time import strftime
import os

In [108]:
API_URL = 'https://api-web.nhle.com/v1'
all_plays = []

In [109]:
def get_matchup_games(start_date, end_date):
    r = requests.get(url=API_URL + '/schedule/' + str(start_date))
    data = r.json()

    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
    matchup_games = {'next_start_date': '', 'game_ids': {'id': [], 'date': []}}

    matchup_games['next_start_date'] = data['nextStartDate']

    for day in data['gameWeek']:
        for game in day['games']:
            # game_date_timestamp = game['startTimeUTC']  # Read the game's start time
            # game_date = datetime.strptime(game_date_timestamp, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d')
            game_date = day['date']
            # Strip the time and retain only the date this causes problems for the sweden games

            if datetime.strptime(game_date, '%Y-%m-%d').date() <= end_date_dt.date():
                matchup_games['game_ids']['id'].append(game['id'])
                matchup_games['game_ids']['date'].append(game_date)

    return matchup_games

In [110]:
def retrieve_schedule(start_date_str, end_date_str):
    all_game_ids = {'game_ids': [], 'game_dates': []}
    temp_week = get_matchup_games(start_date_str, end_date_str)

    all_game_ids['game_ids'].extend(temp_week['game_ids']['id'])
    all_game_ids['game_dates'].extend(temp_week['game_ids']['date'])

    end_date_dt = datetime.strptime(end_date_str, '%Y-%m-%d')

    while True:
        temp_next_start = datetime.strptime(temp_week['next_start_date'], '%Y-%m-%d')

        if temp_next_start <= end_date_dt:
            temp_start = temp_week['next_start_date']
            temp_week = get_matchup_games(temp_start, end_date_str)

            game_ids = temp_week['game_ids']['id']
            game_dates = temp_week['game_ids']['date']

            for game_id, game_date in zip(game_ids, game_dates):
                game_date_dt = datetime.strptime(game_date, '%Y-%m-%d').date()
                if game_date_dt <= end_date_dt.date():
                    all_game_ids['game_ids'].append(game_id)
                    all_game_ids['game_dates'].append(game_date)
                else:
                    # Instead of breaking here, move to the next week
                    break
        else:
            break

    return all_game_ids

In [111]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-10-10'

In [112]:
# Creating a dictionary to store the NHL team information
def get_team_info():
    nhl_teams = {}
    # https://api.nhle.com/stats/rest/en/team
    response = requests.get("https://api.nhle.com/stats/rest/en/team", params={"Content-Type": "application/json"})
    data = response.json()

    for team in data["data"]:
        team_id = team['id']
        team_info = {
            "fullName": team["fullName"],
            "triCode": team["triCode"]
        }
        nhl_teams[team_id] = team_info
    return nhl_teams

In [113]:
team_info = get_team_info()

In [114]:
team_info

{11: {'fullName': 'Atlanta Thrashers', 'triCode': 'ATL'},
 34: {'fullName': 'Hartford Whalers', 'triCode': 'HFD'},
 32: {'fullName': 'Quebec Nordiques', 'triCode': 'QUE'},
 33: {'fullName': 'Winnipeg Jets (1979)', 'triCode': 'WIN'},
 35: {'fullName': 'Colorado Rockies', 'triCode': 'CLR'},
 36: {'fullName': 'Ottawa Senators (1917)', 'triCode': 'SEN'},
 37: {'fullName': 'Hamilton Tigers', 'triCode': 'HAM'},
 38: {'fullName': 'Pittsburgh Pirates', 'triCode': 'PIR'},
 39: {'fullName': 'Philadelphia Quakers', 'triCode': 'QUA'},
 40: {'fullName': 'Detroit Cougars', 'triCode': 'DCG'},
 41: {'fullName': 'Montreal Wanderers', 'triCode': 'MWN'},
 42: {'fullName': 'Quebec Bulldogs', 'triCode': 'QBD'},
 43: {'fullName': 'Montreal Maroons', 'triCode': 'MMR'},
 44: {'fullName': 'New York Americans', 'triCode': 'NYA'},
 45: {'fullName': 'St. Louis Eagles', 'triCode': 'SLE'},
 46: {'fullName': 'Oakland Seals', 'triCode': 'OAK'},
 47: {'fullName': 'Atlanta Flames', 'triCode': 'AFM'},
 48: {'fullName': 

In [115]:
# doesn't include international games
start_2024 = '2024-10-08'
so_far = (retrieve_schedule(start_2024, yesterday))

In [116]:
so_far

{'game_ids': [2024020003,
  2024020004,
  2024020005,
  2024020006,
  2024020007,
  2024020008,
  2024020009,
  2024020010,
  2024020011,
  2024020012,
  2024020013,
  2024020015,
  2024020016,
  2024020014,
  2024020017,
  2024020018,
  2024020019],
 'game_dates': ['2024-10-08',
  '2024-10-08',
  '2024-10-08',
  '2024-10-09',
  '2024-10-09',
  '2024-10-09',
  '2024-10-09',
  '2024-10-09',
  '2024-10-10',
  '2024-10-10',
  '2024-10-10',
  '2024-10-10',
  '2024-10-10',
  '2024-10-10',
  '2024-10-10',
  '2024-10-10',
  '2024-10-10']}

In [117]:
def load_latest_psg_df(directory_path):
    # List all files in the specified directory
    files = os.listdir(directory_path)

    # Filter for files that match the 'psg' pattern with a date suffix
    psg_files = [f for f in files if f.startswith('psg_') and f.endswith('.csv')]

    # Extract the date from the filenames and convert to datetime objects
    dates = []
    for file in psg_files:
        # Extracting date part from the filename (assuming format 'psg_mmddyy.csv')
        date_str = file.split('_')[1].split('.')[0]
        # Parsing the date string to a datetime object
        date = datetime.strptime(date_str, '%m%d%y')
        dates.append((file, date))

    # Find the file with the latest date
    latest_file = max(dates, key=lambda x: x[1])[0]

    # Read the CSV file into a DataFrame
    psg_df = pd.read_csv(os.path.join(directory_path, latest_file))
    return psg_df

In [118]:
# https://api-web.nhle.com/v1/gamecenter/2022030415/play-by-play
def get_livedata_from_game(game_list):
    all_plays = []
    for game in game_list['game_ids']:
        response = requests.get(
            f"{API_URL}/gamecenter/{game}/play-by-play",
            headers={"Content-Type": "application/json"}
        )
        data = response.json()
        temp_game_plays = data.get('plays', [])

        for play in temp_game_plays:
            play_record = {
                'gid': str(game),
                'eventId': play.get('eventId'),
                'sortOrder': play.get('sortOrder'),
                'period_number': play.get('periodDescriptor', {}).get('number'),
                'period_type': play.get('periodDescriptor', {}).get('periodType'),
                'maxRegulationPeriods': play.get('periodDescriptor', {}).get('maxRegulationPeriods'),
                'timeInPeriod': play.get('timeInPeriod'),
                'timeRemaining': play.get('timeRemaining'),
                'situationCode': play.get('situationCode'),
                'homeTeamDefendingSide': play.get('homeTeamDefendingSide'),
                'typeCode': play.get('typeCode'),
                'typeDescKey': play.get('typeDescKey')
            }

            details = play.get('details', {})
            for key, value in details.items():
                play_record[f'details_{key}'] = value

            all_plays.append(play_record)

    return all_plays

In [119]:
# psg_df = get_livedata_from_game(
#     {'game_ids':[2024020019],
#      'game_dates':['2024-10-10']}
# )
psg_df = get_livedata_from_game(so_far)


In [120]:
df = pd.DataFrame(psg_df)

In [121]:
def remove_details_prefix(df: pd.DataFrame) -> pd.DataFrame:
    """
    Renames columns by removing the 'details_' prefix.

    Parameters:
        df (pd.DataFrame): The original DataFrame with columns to rename.

    Returns:
        pd.DataFrame: A DataFrame with renamed columns.
    """
    df = df.rename(columns=lambda x: x.replace('details_', '') if x.startswith('details_') else x)
    return df

In [122]:
df = remove_details_prefix(df)
df.drop(columns=['maxRegulationPeriods', 'timeInPeriod', 'period_type', 'highlightClip',
                 'highlightClipFr','discreteClip','discreteClipFr'], inplace=True)
df.rename(columns={'period_number':'period', 'timeRemaining':'time', 
                   'homeTeamDefendingSide':'homeSide', 'typeCode':'code', 
                   'typeDescKey':'desc', 'eventOwnerTeamId':'team', 'losingPlayerId':'loser',
                   'winningPlayerId':'winner', 'xCoord':'x', 'yCoord':'y',
                   'zoneCode':'zone','shootingPlayerId':'shooter','goalieInNetId':'goalie',
                   }, inplace=True)

# Convert player_id related columns to integers
player_id_columns = ['loser', 'winner', 'hittingPlayerId', 'hitteePlayerId', 'shooter', 'goalie', 
                     'blockingPlayerId', 'playerId', 'scoringPlayerId', 'assist1PlayerId', 
                     'assist2PlayerId', 'committedByPlayerId', 'drawnByPlayerId', 'servedByPlayerId']

for col in player_id_columns:
    if col in df.columns:
        df[col] = df[col].fillna(0).astype(int)

df


,gid,eventId,sortOrder,period,time,situationCode,homeSide,code,desc,team,loser,winner,x,y,zone,hittingPlayerId,hitteePlayerId,shotType,shooter,goalie,awaySOG,homeSOG,reason,blockingPlayerId,playerId,secondaryReason,scoringPlayerId,scoringPlayerTotal,assist1PlayerId,assist1PlayerTotal,awayScore,homeScore,assist2PlayerId,assist2PlayerTotal,code,descKey,duration,committedByPlayerId,drawnByPlayerId,servedByPlayerId
0,2024020003,9,8,1,20:00,1551,left,520,period-start,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
1,2024020003,8,11,1,20:00,1551,left,502,faceoff,19.0,8482665,8480023,0.0,0.0,N,0,0,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
2,2024020003,75,12,1,19:52,1551,left,503,hit,55.0,0,0,93.0,28.0,D,8477955,8476892,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
3,2024020003,78,13,1,19:50,1551,left,503,hit,19.0,0,0,67.0,-40.0,N,8482077,8474586,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
4,2024020003,61,22,1,19:22,1551,left,506,shot-on-goal,19.0,0,0,-61.0,27.0,O,0,0,wrist,8477402,8475831,1.0,0.0,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5511,2024020019,524,831,4,05:00,1331,right,520,period-start,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
5512,2024020019,523,833,4,05:00,1331,right,502,faceoff,19.0,8484801,8480023,0.0,0.0,N,0,0,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
5513,2024020019,1083,836,4,04:15,1331,right,505,goal,19.0,0,0,64.0,20.0,O,0,0,wrist,0,8478406,NaN,NaN,NaN,0,0,NaN,8475170,1.0,8476892,1.0,5.0,4.0,8480023,3.0,NaN,NaN,NaN,0,0,0
5514,2024020019,569,837,4,04:15,1331,right,521,period-end,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0


KeyError: "['maxRegulationPeriods' 'timeInPeriod' 'period_type'] not found in axis"

In [74]:
df.head(50)

,gid,eventId,sortOrder,period,time,situationCode,homeSide,code,desc,team,loser,winner,x,y,zone,hittingPlayerId,hitteePlayerId,shotType,shooter,goalie,awaySOG,homeSOG,reason,blockingPlayerId,playerId,secondaryReason,scoringPlayerId,scoringPlayerTotal,assist1PlayerId,assist1PlayerTotal,awayScore,homeScore,assist2PlayerId,assist2PlayerTotal,code,descKey,duration,committedByPlayerId,drawnByPlayerId,servedByPlayerId
0,2024020003,9,8,1,20:00,1551,left,520,period-start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024020003,8,11,1,20:00,1551,left,502,faceoff,19.0,8482665.0,8480023.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024020003,75,12,1,19:52,1551,left,503,hit,55.0,NaN,NaN,93.0,28.0,D,8477955.0,8476892.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024020003,78,13,1,19:50,1551,left,503,hit,19.0,NaN,NaN,67.0,-40.0,N,8482077.0,8474586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024020003,61,22,1,19:22,1551,left,506,shot-on-goal,19.0,NaN,NaN,-61.0,27.0,O,NaN,NaN,wrist,8477402.0,8475831.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024020003,64,25,1,19:04,1551,left,507,missed-shot,19.0,NaN,NaN,-61.0,-7.0,O,NaN,NaN,wrist,8479385.0,8475831.0,NaN,NaN,hit-crossbar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024020003,70,30,1,18:49,1551,left,506,shot-on-goal,55.0,NaN,NaN,79.0,-30.0,O,NaN,NaN,wrist,8477416.0,8476412.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024020003,76,35,1,18:29,1551,left,507,missed-shot,55.0,NaN,NaN,83.0,9.0,O,NaN,NaN,backhand,8480009.0,8476412.0,NaN,NaN,wide-right,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024020003,96,36,1,18:26,1551,left,503,hit,19.0,NaN,NaN,91.0,-26.0,N,8475170.0,8478840.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024020003,77,37,1,18:13,1551,left,508,blocked-shot,55.0,NaN,NaN,73.0,-9.0,D,NaN,NaN,NaN,8478840.0,NaN,NaN,NaN,blocked,8470600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
